In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df = pd.read_csv("Excels\\diabetes.csv")
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [7]:
df = df.rename(columns={
    'Outcome' : 'Diabetes'
})

In [13]:
df['Diabetes'].value_counts()

Diabetes
0    500
1    268
Name: count, dtype: int64

In [14]:
df.groupby('Diabetes').mean()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
Diabetes,,,,,,,,
0,3.298000,109.980000,68.184000,19.664000,68.792000,30.304200,0.429734,31.190000
1,4.865672,141.257463,70.824627,22.164179,100.335821,35.142537,0.550500,37.067164


In [9]:
df.shape

(768, 9)

In [10]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Diabetes
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [22]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X = df.drop(['Diabetes'],axis='columns')
y = df['Diabetes']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [23]:
X_train , X_test , y_train , y_test = train_test_split(X_scaled,y,test_size=0.2,random_state=20)

In [40]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [41]:
scores = cross_val_score(SVC(),X_scaled,y,cv=5)
scores.mean()

0.7708938120702827

In [42]:
scores = cross_val_score(DecisionTreeClassifier(),X_scaled,y,cv=5)
scores.mean()

0.7071725659960955

In [43]:
scores = cross_val_score(RandomForestClassifier(),X_scaled,y,cv=5)
scores.mean()

0.7644003055767762

In [44]:
model_params = {
    'SVC' :{
        'model' : SVC(),
        'params' : {
            'C' : [1,10,20,50],
            'kernel': ['rbf','linear'],
            'gamma' : ['auto','scale']
        }
    },
    'dec_tree' :{
        'model' : DecisionTreeClassifier(),
        'params' : {
        }
    },
    'rf_clf' :{
        'model' : RandomForestClassifier(),
        'params' : {
            'n_estimators' : [100,200,500]
        }
}
}

In [45]:
scores = []

for model_name,mp in model_params.items():
    clf = GridSearchCV(mp['model'],mp['params'],cv=5)
    clf.fit(X_train,y_train)
    scores.append({
        'model' : model_name,
        'best_score' : clf.best_score_,
        'best_params' : clf.best_params_
    })

df = pd.DataFrame(scores)
df

In [53]:
model = SVC(C=10,gamma='auto',kernel='linear').fit(X_train,y_train)
model.score(X_test,y_test)

0.7597402597402597

In [58]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
accuracy_score(y_pred,y_test)

0.7597402597402597

In [66]:
y_pred

array([0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
      dtype=int64)

In [65]:
for i in range(len(y_pred)):
    if y_pred[i] == 1:  # Assuming 1 represents 'Diabetes' class
        print('Have Diabetes')
    else:
        print("Don't have Diabetes")


Don't have Diabetes
Have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Have Diabetes
Don't have Diabetes
Have Diabetes
Have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Have Diabetes
Have Diabetes
Have Diabetes
Don't have Diabetes
Have Diabetes
Don't have Diabetes
Have Diabetes
Have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Have Diabetes
Have Diabetes
Don't have Diabetes
Have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Have Diabetes
Don't have Diabetes
Don't have Diabetes
Don't have Diabetes
Have Diabetes
Don't have Diabetes
Have Diabetes
Don't ha